In [1]:
import re
import os
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns

In [2]:
sn_df = pd.read_csv("Social_Networks_data.csv")
ns_df = pd.read_csv("Network_Science_data.csv")
jcn_df = pd.read_csv("Journal_of_Complex_Networks_data.csv")

In [3]:
sn_df.columns

Index(['Document Type', 'Authors', 'Title', 'Journal', 'Year', 'Citations',
       'Keywords', 'DOI', 'Affiliations', 'References'],
      dtype='object')

In [4]:
sn_df = pd.read_csv("Social_Networks_data.csv")
drop_index = []
for index,row in sn_df.iterrows():
    if len(eval(row.iloc[1])) == 0:
        drop_index.append(index)
        continue
    sn_df.iloc[index, 1] = eval(row.loc["Authors"])[0]
sn_df.drop(drop_index, axis=0, inplace=True)
sn_df.reset_index(drop=True, inplace=True)

In [5]:
# Function to calculate bibliometric metrics
def calculate_metrics(df, journal_name):
    # Ensure Citations column is numeric
    df['Citations'] = pd.to_numeric(df['Citations'], errors='coerce').fillna(0).astype(int)

    # **1. Number of Cited Publications (NCP)**
    NCP = (df['Citations'] > 0).sum()

    # **2. Proportion of Cited Publications (PCP)**
    PCP = round(NCP / len(df), 4) if len(df) > 0 else 0

    # **3. Citations per Cited Publication (CCP)**
    CCP = round(df[df['Citations'] > 0]['Citations'].mean(), 2) if NCP > 0 else 0

    # **4. h-index**
    def h_index(citations):
        citations_sorted = sorted(citations, reverse=True)
        return sum(c >= i+1 for i, c in enumerate(citations_sorted))

    h_index_value = h_index(df['Citations'])

    # **5. g-index**
    def g_index(citations):
        citations_sorted = sorted(citations, reverse=True)
        g = 0
        while g < len(citations_sorted) and sum(citations_sorted[:g+1]) >= (g+1)**2:
            g += 1
        return g

    g_index_value = g_index(df['Citations'])

    # **6. i-index (i-10, i-100, i-200)**
    i_10 = (df['Citations'] >= 10).sum()
    i_100 = (df['Citations'] >= 100).sum()
    i_200 = (df['Citations'] >= 200).sum()

    # Store results in a dictionary
    return {
        "Journal": journal_name,
        "NCP": NCP,
        "PCP": PCP,
        "CCP": CCP,
        "h-index": h_index_value,
        "g-index": g_index_value,
        "i-10 index": i_10,
        "i-100 index": i_100,
        "i-200 index": i_200
    }

# Calculate metrics for each journal
metrics_list = [
    calculate_metrics(sn_df, "Social Networks"),
    calculate_metrics(ns_df, "Network Science"),
    calculate_metrics(jcn_df, "Journal of Complex Networks")
]

# Convert results to DataFrame
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv("Journal Metrics.csv")


In [17]:
print(metrics_df.to_latex())

\begin{tabular}{llrrrrrrrr}
\toprule
 & Journal & NCP & PCP & CCP & h-index & g-index & i-10 index & i-100 index & i-200 index \\
\midrule
0 & Social Networks & 1462 & 0.939000 & 64.510000 & 126 & 264 & 1004 & 170 & 69 \\
1 & Network Science & 252 & 0.834400 & 14.680000 & 23 & 52 & 78 & 6 & 1 \\
2 & Journal of Complex Networks & 431 & 0.786500 & 18.500000 & 34 & 78 & 138 & 10 & 5 \\
\bottomrule
\end{tabular}

